In [147]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split 

In [157]:
df = pd.read_csv('data/first_round_training_data.csv')

In [158]:
df = df.drop(['Attribute'+str(i+1) for i in range(10)], axis=1)

In [159]:
df['Quality_label'] = pd.Categorical(df['Quality_label'])
df['Quality_label'] = df.Quality_label.cat.codes

In [160]:
column_names = ['Parameter'+str(i+1) for i in range(10)]+['Quality_label']

In [161]:
df.head()

,Parameter1,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Parameter10,Quality_label
0,0.001660,0.591013,147.608373,38.186345,0.000421,0.000612,2286.523413,0.035407,0.593081,1.010385,3
1,1.601749,0.015052,0.035864,51.130326,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,1
2,0.098039,69.233685,0.080920,0.112265,0.000909,0.001972,2286.523413,0.035407,0.593081,1.010385,1
3,18.181860,0.047325,0.018061,1.098102,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,1
4,0.012085,0.008749,0.005509,524.327396,0.000909,0.002397,2286.523413,0.035407,0.593081,1.010385,1


In [162]:
df.dtypes

Parameter1       float64
Parameter2       float64
Parameter3       float64
Parameter4       float64
Parameter5       float64
Parameter6       float64
Parameter7       float64
Parameter8       float64
Parameter9       float64
Parameter10      float64
Quality_label       int8
dtype: object

In [163]:
# sns.pairplot(train_dataset[column_names], diag_kind="kde")

In [164]:
# df = df[df['Parameter1']<=200000]
# df = df[df['Parameter2']<=500000]
# df = df[df['Parameter3']<=200000]
# df = df[df['Parameter4']<=50000]
# df = df[df['Parameter5']<=60]
# df = df[df['Parameter6']<=50]
# df = df[df['Parameter7']<=10000]
# df = df[df['Parameter8']<=5000]
# df = df[df['Parameter9']<=1]
# df = df[df['Parameter10']<=1000]
# sns.pairplot(df[column_names], diag_kind="kde")

In [165]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

3840 train examples
960 validation examples
1200 test examples


In [172]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('Quality_label')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

batch_size = 16
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of Parameter1:', feature_batch['Parameter1'])
    print('A batch of Quality_label:', label_batch )

Every feature: ['Parameter1', 'Parameter2', 'Parameter3', 'Parameter4', 'Parameter5', 'Parameter6', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10']
A batch of Parameter1: tf.Tensor(
[6.6967779e-01 3.8130215e-01 2.2905863e+01 4.3957529e+02 1.2992480e+00
 2.7035661e+00 1.7333443e+00 3.0718412e+00 1.4833006e-01 1.8765192e+00
 4.4078326e-01 1.7793991e-01 1.2000130e-01 1.5603722e+01 1.6469117e+00
 7.8751557e-02], shape=(16,), dtype=float32)
A batch of Quality_label: tf.Tensor([3 3 3 2 1 0 1 3 3 3 0 0 3 2 2 1], shape=(16,), dtype=int32)


In [134]:
# dataset = df.copy()
# train_dataset = dataset.sample(frac=0.8,random_state=0)
# test_dataset = dataset.drop(train_dataset.index)
# train_labels = train_dataset.pop('Quality_label')
# test_labels = test_dataset.pop('Quality_label')

In [174]:
train_stats = train.describe()
# train_stats.pop("Quality_label")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Parameter1,3840.0,477.512753,1.200014e+04,2.983955e-06,0.089863,1.082774,11.031183,7.136822e+05
Parameter2,3840.0,1632.170510,4.989668e+04,1.683797e-07,0.049703,0.995893,15.862083,3.034568e+06
Parameter3,3840.0,461.438690,1.463083e+04,5.043728e-06,0.096786,1.084830,10.643703,8.645306e+05
Parameter4,3840.0,174.546366,2.042208e+03,1.617118e-06,0.116415,1.066520,9.948704,8.831222e+04
Parameter5,3840.0,5.998526,8.839949e+00,9.985942e-07,0.741630,2.225808,6.680183,8.367195e+01
Parameter6,3840.0,5.636587,7.767213e+00,9.810610e-07,0.381815,2.684398,7.117767,4.117541e+01
Parameter7,3840.0,245.930299,1.895501e+03,6.238222e-04,0.152059,0.600827,2.374035,3.569859e+04
Parameter8,3840.0,15.647271,9.360098e+01,2.369918e-02,0.035407,2.931083,17.850021,1.208918e+03
Parameter9,3840.0,45699.108036,2.804895e+06,3.962831e-04,0.593081,0.593081,6.783967,1.738083e+08
Parameter10,3840.0,34.608217,9.922221e+01,1.023626e-03,0.101478,0.727611,13.969790,5.171976e+02


In [176]:
# def norm(x):
#     return (x - train_stats['mean']) / train_stats['std']
# train_ds = norm(train_ds)
# val_ds = norm(val_ds)
# test_ds = norm(test_ds)

In [141]:
final_dataset = tf.data.Dataset.from_tensor_slices((normed_train_data.values, train_labels.values))

In [144]:
final_dataset

<TensorSliceDataset shapes: ((10,), ()), types: (tf.float64, tf.int8)>

In [177]:
def build_model():
    model = keras.Sequential([
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(50, activation='relu'),
        keras.layers.Dense(20, activation='relu'),
        keras.layers.Dense(4, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_model()

In [178]:
EPOCHS = 10

model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

ValueError: Passing a dictionary input to a Sequential Model which doesn't have FeatureLayer as the first layer is an error.

In [ ]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)